## I. Load & Process the data from Sleeper

In [1]:
from sleeper_functions import *
#Define week
week = 2
matchup_df, updated_standings, HT_Owner, HT_Score, HP_Owner, HP_Player, HP_Score = run_sleeper_weekly(week)

Loaded the League Data from Sleeper
 - Week 1 has been processed
 - Week 2 has been processed
Completed processing scores and updating standings
Saved week 2 matchup data


## II. Feed Data into GPT

In [25]:
summary = generate_summary(week, matchup_df, updated_standings)
print(summary)

Hey there, 

Week 2 in our one-year-old Dynasty league is officially in the books, and boy, was it a doozy! We had some high-scoring matchups, some underperforming teams, and some, well, unfortunate sounding team names. Let's dive right into the action!

In the highlight matchup of the week, Andrew's "King Drew" continued his dominance by taking down JD's "Big Cheese." Andrew rode the arm of Daniel Jones, who somehow managed to put up 31.74 points (I didn't know he had that in him either) and Tyreek Hill's steady performance to secure a 121.08-109.54 victory. JD, on the other hand, couldn't quite keep up despite strong showings from Bijan Robinson and Derrick Henry. It seems the "Big Cheese" may have melted a bit this week.

Johnny's "ChubbHub" had a tough go of it as he fell to David's "Korean Jesus" in a battle that went down to the wire. Despite strong performances from Jared Goff and Saquon Barkley, Johnny couldn't overcome a receiving corps led by Justin Jefferson and a solid outi

## III. Draft Summary Email via Smtpblib

In [27]:
#Load Credentials
import json
# Load the configuration from config.json
with open('config.json') as config_file:
    config_data = json.load(config_file)
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Load the user name and password from yaml file
user, password = config_data['gmail']['GMAIL_USER'], config_data['gmail']['GMAIL_PW']

matchup = matchup_df.to_html(index=False)
standings = updated_standings.to_html(index=False)

# Initialize the server
s = smtplib.SMTP('smtp.gmail.com', 587)
s.starttls()
s.login(user, password)

# Define message
subject = f"Dynasty FF Week {week} Recap"
summary_html = summary.replace('\n', '<br>')
body = f"""\
<html>
  <head></head>
  <body>
    <h1>{subject}</h1>
    <br>{summary_html}</br>
    <br>
    <hr>
    <p>Highest Scoring Team: {HT_Owner}: {HT_Score} points</p>
    <p>Highest Scoring Player: {HP_Owner} - {HP_Player}: {HP_Score} Points</p>
    <br>
    {standings}
  </body>
</html>
"""

recipients = list(config_data['emails']['sleeper_email_dict'].values())
sender = "peteryushunli@gmail.com"

msg = MIMEMultipart("alternative")
msg["From"] = sender
msg["To"] = ", ".join(recipients)
msg["Subject"] = subject

# Attach the HTML body
msg.attach(MIMEText(body, 'html'))

# Send email
s.sendmail(sender, recipients, msg.as_string())

# Terminate the session
s.quit()

(221,
 b'2.0.0 closing connection d3-20020a0caa03000000b00655e3e86ad7sm4834421qvb.6 - gsmtp')